## Demonstration of orbit fitting for the LMC–Milky Way interaction

In [ ]:
import agama, numpy, scipy.optimize, scipy.interpolate, scipy.integrate, matplotlib.pyplot as plt
%matplotlib inline
numpy.set_printoptions(linewidth=1000, precision=6, suppress=True)
agama.setUnits(length=1, velocity=1, mass=1)  # 1 kpc, 1 km/s, 1 Msun

### Routines for constructing a smooth approximation to the orbits of MW and LMC extracted from N-body simulations

In [ ]:
def getBsplineValuesAndDerivs(degree, knots, x):
    """
    Construct matrices of values and derivatives of B-splines
    of a given degree and knots at the given array of points x.
    Arguments:
        degree >= 1:  degree of B-splines;
        knots:  array of knots (without repeating the endpoints);
        x:  array of coordinates where the values and derivs should be computed.
    Return:
        two arrays of shape (len(x), Nbasis), where Nbasis=len(knots)+degree-1,
        containing the values and derivatives of Nbasis B-spline functions at x.
    """
    # scipy B-splines operate on an extended array of knots, where the endpoints
    # are repeated (degree+1) times. Moreover, we need to shift the rightmost point
    # a tiny bit higher, because if x==knots[-1] exactly, scipy would consider it
    # to fall outside the grid and return zeros, but we mean it to be within the grid.
    bknots = numpy.hstack([[knots[0]] * degree, knots[:-1],
                           [numpy.nextafter(knots[-1], numpy.inf)] * (degree+1)])
    vals = numpy.zeros((len(x), len(knots)+degree-1))
    ders = numpy.zeros((len(x), len(knots)+degree-1))
    for p in range(len(knots)+degree-1):
        vals[:,p] = numpy.nan_to_num(
            scipy.interpolate.BSpline.basis_element(bknots[p:p+degree+2], False)(x) )
        if p==0: continue
        der = numpy.nan_to_num(
            scipy.interpolate.BSpline.basis_element(bknots[p:p+degree+1], False)(x) *
            degree / (bknots[p+degree] - bknots[p]))
        ders[:,p-1] -= der
        ders[:,p  ] += der
    return vals, ders

In [ ]:
def fitSmoothOrbit(tgrid, trajm, trajl, potm, potl):
    """
    Construct a smooth approximation to the orbits of MW and LMC extracted from the N-body simulation.
    The assumption is that the trajectories of both galaxies satisfy the following ODE system:
      dx_m / dt = v_m
      dx_l / dt = v_l
      dv_m / dt = -\grad\Phi_l(x_m-x_l) + a_m
      dv_l / dt = -\grad\Phi_m(x_l-x_m) + a_l
    where x_m, v_m, x_l, v_l are the positions and velocities of MW and LMC,
    \Phi_m, \Phi_l are their gravitational potentials (assumed to be non-deforming),
    and a_m, a_l are the additional accelerations not accounted for in this model
    (principally, dynamical friction, but also self-friction, forces from deformations, etc.).
    The additional accelerations are assumed to be much smaller than the gradients of Phi,
    and vary on a longer timescale than \grad\Phi (the latter change rapidly near the pericentre).
    In addition, the measured trajectories in the N-body simulation contain some amount of noise
    both in position and velocity of both galaxies.
    The fitting procedure approximates the noisy trajectories of galaxies by solutions of this ODE
    and simultaneously determines the additional accelerations a_l, a_m(t) that remove the large-scale
    residuals in this fit, but vary on sufficiently long timescales to ignore the noise.
    These additional accelerations are represented by B-splines with a small number of nodes,
    whose coefficients are fitted in the least-square sense to the measured x_m, v_m, x_l, v_l.
    Arguments:
        tgrid:  timestamps for the trajectories (length N);
        trajm:  measured trajectory of the MW  (shape: (N,6));
        trajl:  measured trajectory of the LMC (shape: (N,6));
        potm:   the gravitational potential of the MW;
        potl:   the gravitational potential of the LMC.
    Return:
        a tuple of two arrays of the same shape (N,6) that represent the smooth approximations
        to trajm, trajl.
    """
    degree = 5  # correcting splines of degree 5 for position, 4 for velocity, 3 for acceleration
    # construct a suitable time grid for B-splines, denser around the pericentre(s)
    r  = numpy.sum((trajl-trajm)[:,0:3]**2, axis=1)**0.5
    dt = (r/600)**1.0  # approximate time interval between grid nodes (chosen by trial and error)
    cs = numpy.hstack([0, numpy.cumsum((tgrid[1:] - tgrid[:-1]) / (dt[1:] + dt[:-1])) ])
    # put an integer number of nodes equally spaced in cumsum, with first and last nodes exactly at endpoints
    inds = numpy.searchsorted(cs, numpy.linspace(0, cs[-1], round(cs[-1])))
    knots = tgrid[inds]
    #print(knots)
    # the extended array of spline knots for scipy.BSpline (see the explanation in getBsplineValuesAndDerivs)
    bknots = numpy.hstack([[knots[0]] * degree, knots[:-1],
                           [numpy.nextafter(knots[-1], numpy.inf)] * (degree+1)])
    matrixpos, matrixvel = getBsplineValuesAndDerivs(degree, knots, tgrid)
    tnorm  = 0.5  # dimensional factor to bring velocity on the same scale as position in the fit
    # we fit the position and velocity of each galaxy simultaneously
    matrix = numpy.vstack([matrixpos, tnorm * matrixvel])

    # since the RHS of the ODE contains accelerations computed along the trajectory, and we want to use
    # smoothed rather than actual trajectories, this procedure is iterative (two iterations is enough)
    trajmfit = trajm.copy()
    trajlfit = trajl.copy()
    for iteration in range(2):
        # the main terms in the RHS of the ODE are the accelerations from gravitational potentials
        accm = potl.force(trajmfit[:,0:3] - trajlfit[:,0:3])  # -\grad\Phi_l(x_m - x_l)
        accl = potm.force(trajlfit[:,0:3] - trajmfit[:,0:3])  # -\grad\Phi_m(x_l - x_m)
        for c in range(3):
            # spline representing the acceleration of MW from the potential of the LMC
            sacc = scipy.interpolate.make_interp_spline(tgrid, accm[:,c], degree-2, bc_type=('natural', 'natural'))
            # integrate it twice to obtain the velocity and position,
            # which would be the solutions of the ODE in absense of additional accelerations a_l,a_m
            svel = sacc.antiderivative()
            spos = svel.antiderivative()
            trajmfit[:,c]   = spos(tgrid)
            trajmfit[:,c+3] = svel(tgrid)
            # the actual trajectory deviates from the expectation because of the additional acceleration a_m,
            # whose cumulative effect is represented as a B-spline with coefficients that are free parameters
            # in the least-square fit to the deviation in position (spline itself) and in velocity (its derivative).
            rhs  = numpy.hstack([trajm[:,c] - trajmfit[:,c], (trajm[:,c+3] - trajmfit[:,c+3]) * tnorm])
            coefs= numpy.linalg.lstsq(matrix, rhs, rcond=None)[0]
            # construct the correction spline for the position
            bspl = scipy.interpolate.BSpline(bknots, coefs, degree)
            # and add it to the trajectory derived from the ODE
            # (values of the spline itself for position, its derivatives for velocity).
            trajmfit[:,c]   += bspl(tgrid)
            trajmfit[:,c+3] += bspl(tgrid, 1)
            
            # same procedure for the LMC trajectory
            sacc = scipy.interpolate.make_interp_spline(tgrid, accl[:,c], 3, bc_type=('natural', 'natural'))
            svel = sacc.antiderivative()
            spos = svel.antiderivative()
            trajlfit[:,c]   = spos(tgrid)
            trajlfit[:,c+3] = svel(tgrid)
            rhs  = numpy.hstack([trajl[:,c]-trajlfit[:,c], (trajl[:,c+3]-trajlfit[:,c+3]) * tnorm])
            coefs= numpy.linalg.lstsq(matrix, rhs, rcond=None)[0]
            bspl = scipy.interpolate.BSpline(bknots, coefs, degree)
            trajlfit[:,c]   += bspl(tgrid)
            trajlfit[:,c+3] += bspl(tgrid, 1)

    return trajmfit, trajlfit

In [ ]:
def buildOrbit(tgrid, traj):
    """
    Construct a callable object that interpolates the trajectory as a function of time.
    Arguments:
        tgrid:  timestamps for the trajectory (length N)
        traj:   trajectory (position and velocity, shape (N,6))
    Return:
        a function with signature
            fnc(t, acc=False)
        whose argument `t` is a single number or an array of N times,
        and return value is a Nx6 array of positions and velocities (if acc=False),
        or Nx9 array of positions, velocities and accelerations (if acc=True)
        at the given timestamps.
        The solution is interpolated as a 5th degree B-spline for position,
        4th degree for velocity and 3th degree for acceleration;
        it can be extrapolated beyond the originally provided interval tgrid,
        but the accuracy of this extrapolation is rapidly deteriorating
        as one gets further away from the endpoints.
    """
    # we actually do *not* use velocities from the provided array, except at the two endpoints,
    # and set the boundary conditions at these endpoints as follows: first derivative is the velocity,
    # fourth derivative is zero (same convention as quintic splines from Agama).
    x,y,z = [scipy.interpolate.make_interp_spline(tgrid, traj[:,c], 5,
       bc_type=([(1, traj[0,c+3]), (4, 0.0)], [(1, traj[-1,c+3]), (4, 0.0)]) ) for c in range(3)]
    def sol(t, acc=False):
        if acc:
            result = numpy.column_stack([x(t), y(t), z(t), x(t,1), y(t,1), z(t,1), x(t,2), y(t,2), z(t,2)])
        else:
            result = numpy.column_stack([x(t), y(t), z(t), x(t,1), y(t,1), z(t,1)])
        # if the input was a single point (not an array), we return a 1d array, otherwise a 2d array
        try:
            return result.reshape(len(t), -1)
        except TypeError:
            return result.reshape(-1)
    return sol

In [ ]:
def checkSmoothOrbit(axes, tgrid, trajm, trajl, trajmfit, trajlfit, potm, potl):
    """
    Produce diagnostic plots comparing the N-body trajectories with the smoothed ones.
    Arguments:
        axes:  array of three matplotlib axes objects for plotting position, velocity and acceleration;
        tgrid:  timestamps for the trajectory
        trajm, trajl:  measured trajectories of MW and LMC from the N-body simulation;
        trajmfit, trajlfit:  approximate trajectories fitted by fitSmoothOrbit;
        potm, potl:  potentials of MW and LMC
    Return:
        nothing, but the axes will be filled with the following plots:
        axes[0]: difference between measured and fitted positions of MW (solid lines)
        and LMC (dashed lines) as functions of time;
        axes[1]: same for the velocity;
        axes[2]: total acceleration of both galaxies computed from smooth orbits;
        axes[3]: difference between the total accelerations determined from the smooth orbits and
        the expected values from the analytic potentials of both galaxies (i.e. the additional
        accelerations responsible for dynamical friction, etc., derived during the fit);
        solid lines for MW, dashed for LMC.
    """
    taccm = buildOrbit(tgrid, trajmfit)(tgrid, acc=True)[:,6:9]  # measured total accelerations
    taccl = buildOrbit(tgrid, trajlfit)(tgrid, acc=True)[:,6:9]
    paccm = potl.force(trajmfit[:,0:3] - trajlfit[:,0:3])  # accelerations expected from potentials alone
    paccl = potm.force(trajlfit[:,0:3] - trajmfit[:,0:3])
    colorm= ['b', 'g', 'r']
    colorl= ['#a0a000', 'm', '#00a0a0']
    labels= ['x', 'y', 'z']
    imin, imax = numpy.searchsorted(tgrid, [-0.25, 0.25])
    for c in range(3):
        axes[0].plot(tgrid, trajm[:,c  ]-trajmfit[:,c  ], color=colorm[c], lw=0.5, label='MW '+labels[c])
        axes[1].plot(tgrid, trajm[:,c+3]-trajmfit[:,c+3], color=colorm[c], lw=0.5, label='MW '+labels[c])
        axes[2].plot(tgrid, taccm[:,c  ]                , color=colorm[c], lw=0.5, label='MW '+labels[c])
        axes[3].plot(tgrid, taccm[:,c  ]-paccm   [:,c  ], color=colorm[c], lw=0.5, label='MW '+labels[c])
    for c in range(3):
        axes[0].plot(tgrid, trajl[:,c  ]-trajlfit[:,c  ], color=colorl[c], lw=0.7, label='LMC '+labels[c], dashes=[4,3])
        axes[1].plot(tgrid, trajl[:,c+3]-trajlfit[:,c+3], color=colorl[c], lw=0.7, label='LMC '+labels[c], dashes=[4,3])
        axes[2].plot(tgrid, taccl[:,c  ]                , color=colorl[c], lw=0.7, label='LMC '+labels[c], dashes=[4,3])
        axes[3].plot(tgrid, taccl[:,c  ]-paccl   [:,c  ], color=colorl[c], lw=0.7, label='LMC '+labels[c], dashes=[4,3])
    for ax in axes:
        #ax.legend(loc='upper left')
        ax.set_xlim(min(tgrid), max(tgrid))
        ax.set_xlabel('time')
    '''axes[0].text(0.98, 0.04, r'$\delta x_{\rm MW}=%.3f$, $\delta x_{\rm LMC}=%.3f$' % (
                 numpy.mean((trajm-trajmfit)[imin:imax,0:3]**2)**0.5,
                 numpy.mean((trajl-trajlfit)[imin:imax,0:3]**2)**0.5),
                 ha='right', va='bottom', transform=axes[0].transAxes)
    axes[1].text(0.98, 0.04, r'$\delta v_{\rm MW}=%.3f$, $\delta v_{\rm LMC}=%.3f$' % (
                 numpy.mean((trajm-trajmfit)[imin:imax,3:6]**2)**0.5,
                 numpy.mean((trajl-trajlfit)[imin:imax,3:6]**2)**0.5),
                 ha='right', va='bottom', transform=axes[1].transAxes)'''
    axes[0].set_ylabel('delta position')
    axes[1].set_ylabel('delta velocity')
    axes[2].set_ylabel('total acceleration')
    axes[3].set_ylabel('additional acceleration')


### Nonlinear coordinate transformation

In [ ]:
class Basis:
    """
    The class implementing the coordinate transformations for phase-space points
    defined by a reference point P0.
    There are two (and a half) key ingredients for success.
    The first "half" is a preliminary step to bring the velocity on the same scale as position,
    by defining a suitable scaling factor "tnorm" with the dimension of time.
    Then we construct an orthonormal basis in the scaled 6d phase space, which is rotated w.r.t.
    (x, y, z, vx*tnorm, vy*tnorm, vz*tnorm) and instead aligns with the orbital tori, loosely speaking.
    Namely, the 0th axis of the new basis points in the direction of motion (dx/dt, tnorm*dv/dt) at P0,
    where the acceleration dv/dt is assumed to be created by the MW+LMC potentials (which is a sensible
    approximation, but does not take into account dynamical friction, etc. - so that the actual motion
    followed by the reference point P0 may slightly deviate from this direction).
    Nevertheless, the hyperplane orthogonal to this 0th axis is likely crossed by all nearby orbits,
    so the position in this fiducial hyperplane is a proxy of how much a given point P deviates from
    the reference point P0 in space, and the displacement along the 0th axis describes its deviation
    in time (positive when the point P is "ahead" of the reference point P0).
    These steps alone define a linear transformation (rotation) of the phase space, so would not be
    sufficient in the case of rapidly varying motions near the pericentre.
    Therefore, a second key idea is that any point P in the rotated 6d phase space can be tracked back
    the 5d hyperplane defined above, along a nonlinear trajectory. The intersection of this trajectory
    with the plane P' defines the 5 components of deviation vector, and the remaining 0th component is
    the time offset between the points P and P' (positive if the point P is ahead of P').
    To execute this nonlinear transformation, we need to know not only the point P, but the entire
    trajectory of this point over some interval of time that crosses the fiducial hyperplane.
    The transformation from the orbit to the nonlin-scaled point amounts to finding the moment of time
    where the orbit crosses the hyperplane (i.e. the 0th component of the point in the rotated basis
    changes sign). The inverse transformation from a nonlin-scaled point to the actual phase space
    amounts to predicting the trajectory, which can be done only approximately - assuming that it
    follows Hamilton's equations for the given potential (again, this ignores dynamical friction and
    other non-conservative forces, but this approximation should be good enough far from pericentre).
    """
    def __init__(self, time, point, potm, potl):
        """
        Construct the basis for the given reference phase-space point P0 at a given time.
        Arguments:
            time:   reference value of time corresponding to the point P0;
            point:  6d phase-space point P0 (relative position and velocity of LMC and MW);
            potm, potl:  potentials of MW and LMC.
        """
        point = numpy.array(point)
        pos0  = point[0:3]
        vel0  = point[3:6]
        # expected dv/dt from the potential gradient
        acc0  = potm.force(pos0[0:3], t=time) - potl.force(-pos0[0:3], t=time)
        # dimensional factor [time] for putting the velocity on the same scale as position
        self.tnorm = sum(vel0**2)**0.5 / sum(acc0**2)**0.5  # not necessarily optimal, but a plausible choice
        self.tnorm = 1.0  # another quite arbitrary but plausible choice, seems to do better than the above..
        # the first two basis vectors are chosen as follows:
        # vec0 points in the direction of motion (dx/dt, dv/dt);
        # vec1 points in the direction opposite to the gradient of the Hamiltonian (-dPhi/dx, -v);
        vec0  = numpy.hstack([vel0, acc0 * self.tnorm])
        vec0 /= sum(vec0**2)**0.5
        vec1  = numpy.hstack([acc0 * self.tnorm, -vel0])
        vec1 /= sum(vec1**2)**0.5
        # vec2 points in the direction opposite to the gradient of the total angular momentum (-dL^2/dx, -dL^2/dv)
        Lx = pos0[1] * vel0[2] - pos0[2] * vel0[1]
        Ly = pos0[2] * vel0[0] - pos0[0] * vel0[2]
        Lz = pos0[0] * vel0[1] - pos0[1] * vel0[0]
        vec2  = -numpy.array([self.tnorm, self.tnorm, self.tnorm, 1, 1, 1]) * numpy.array([
            Lz*vel0[1]-Ly*vel0[2], Lx*vel0[2]-Lz*vel0[0], Ly*vel0[0]-Lx*vel0[1],
            Ly*pos0[2]-Lz*pos0[1], Lz*pos0[0]-Lx*pos0[2], Lx*pos0[1]-Ly*pos0[0] ])
        # because vec1 also had some component along this gradient of angular momentum,
        # vec2 must be made orthogonal to the previous two vectors (i.e. it now points in the direction
        # of the fastest change of angular momentum that does not change the energy)
        vec2 -= vec0*(vec0.dot(vec2))
        vec2 -= vec1*(vec1.dot(vec2))
        vec2 /= sum(vec2**2)**0.5  # and now it is normalized as well
        # vec3 is constructed from vec2 in the same way as vec1 from vec0
        # (swapping the first and second triplets with the change of sign in one of them)
        vec3  = numpy.hstack([vec2[3:6], -vec2[0:3]])
        # for the last two vectors, we use a completely fudged up approach, which nevertheless gives sensible results
        vec4  = numpy.array([
            vec0[1]*vec2[2]-vec0[2]*vec2[1], vec0[2]*vec2[0]-vec0[0]*vec2[2], vec0[0]*vec2[1]-vec0[1]*vec2[0],
            vec1[1]*vec3[2]-vec1[2]*vec3[1], vec1[2]*vec3[0]-vec1[0]*vec3[2], vec1[0]*vec3[1]-vec1[1]*vec3[0] ])
        vec4 -= vec1*(vec1.dot(vec4))
        vec4 -= vec3*(vec3.dot(vec4))
        vec4 /= numpy.sum(vec4**2)**0.5
        vec5  = numpy.hstack([vec4[3:6], -vec4[0:3]])
        self.basis = numpy.column_stack([vec0, vec1, vec2, vec3, vec4, vec5])
        # check that the above procedure did produce an orthonormal basis
        assert numpy.max(abs(self.basis.T.dot(self.basis) - numpy.eye(6))) < 1e-14
        self.time   = time
        self.origin = point
        self.potm   = potm
        self.potl   = potl

    def torotated(self, point):
        """
        Convert from physical position/velocity to coordinates in the rotated phase space.
        Argument:
            point:  a single vector of length 6 (pos, vel) or an array of shape (N,6).
        Return:
            array of the same shape - input points represented in this rotated basis.
        """
        a = (point - self.origin).T
        a[3:6] *= self.tnorm
        return self.basis.T.dot(a).T

    def fromrotated(self, rotated_point):
        """
        Convert from point(s) in the rotated phase space back to physical position/velocity.
        The transformation is exactly opposite to `torotated` (up to machine precision).
        Argument:
            rotated_point:  a single vector of length 6 or an array of shape (N,6).
        Return:
            array of the same shape - input points in the physical pos/vel space.
        """
        a = self.basis.dot(rotated_point.T)
        a[3:6] /= self.tnorm
        return (a.T + self.origin)

    def toscaled(self, orbit):
        """
        Convert from a trajectory (not a single point!) to the nonlinearly scaled point.
        Argument:
            orbit:  a callable object returning the 6d position/velocity as a function of time.
        Return:
            a vector of length 6, whose 0th component is the time offset between the input orbit
            and the reference point (P0,t0) used to construct this basis (positive offset means
            that the orbit is "ahead of" the point P0, i.e. it was crossing the reference plane
            at an earlier time than t0), and the remaining 5 components give the position of
            the crossing point in the reference hyperplane.
            If the crossing cannot be determined, return a vector of NaNs.
        """
        # first coarsely locate the interval containing the root
        ta = self.time
        while not self.torotated(orbit(ta))[0] < 0 and ta>-10:
            ta -= 0.25
        tb = ta + 0.25
        while not self.torotated(orbit(tb))[0] > 0 and tb<10:
            tb += 0.25
        try:
            # now find the time t0 on the interval ta..tb where the orbit crosses the hyperplane
            t0 = scipy.optimize.brentq(lambda t: self.torotated(orbit(t))[0], ta, tb)
            # corresponding point in the hyperplane, with 0th coordinate being zero
            p0 = self.torotated(orbit(t0))
            # put the time offset into the 0th element of the nonlin-scaled vector:
            # if t0 < self.time, the input point was "ahead of" the reference point,
            # so the time offset is positive, and vice versa
            return numpy.hstack([self.time-t0, p0[1:6]])
        except ValueError:
            # shouldn't happen since we have already bracketed root - might be some other problem
            return numpy.zeros(6) * numpy.nan

    def fromscaled(self, scaled_point):
        """
        Convert from a nonlinearly scaled point to the physical position/velocity space.
        Argument:
            scaled_point:  a vector of length 6, whose 0th element is a time offset and the
            remaining 5 components are the coordinates in the reference hyperplane.
        Return:
            a vector of length 6 that is the expected position/velocity of the input point
            along a trajectory started in the reference hyperplane and integrated for
            for the duration of time given in its 0th element.
            In general, this is not an exact inverse operation to `toscaled`, 
            but it would have been exact if the orbit provided to that function followed
            the Hamilton equations of motion in the given MW+LMC potential.
        """
        # first establish the point in the reference hyperplane and convert it to physical space
        ic = self.fromrotated(numpy.hstack([0, scaled_point[1:6]]))
        # then integrate the trajectory (approximately, assuming Hamiltonian equations of motion)
        # from this point for a given interval of time (stored in the 0th component of nonlinpoint)
        deltat = scaled_point[0]
        if deltat == 0:
            return ic
        def difeq(vars, t):
            return numpy.hstack([vars[3:6], self.potm.force(vars[0:3]) - self.potl.force(-vars[0:3])])
        return scipy.integrate.odeint(difeq, ic, [0, deltat])[1]  # take the final point from the integration

### The core routine

In [ ]:
def simulateOrbit(Tcurr, Tbegin, ic0, potm, potl):
    '''
    The core function for shooting a bunch of orbits,
    computing the deviations of final coordinates from the target,
    and determining the next IC.
    The actual simulations are performed by a separate routine, getTrajectories,
    which should take an array of ICs for a bunch of orbits, run the sims,
    and report measured (noisy) trajectories of both galaxies.
    Arguments:
        Tcurr:  current time corresponding to the final coordinates (posvelLMC)
        Tbegin: initial time for the simulations
        ic0:    initial conditions for the base orbit
        potm:   Milky Way potential
        potl:   LMC potential
    Return:
        two possible suggestions for the next IC from different methods;
        also shows a diagnostic plot explained in the code comments.
    '''
    fcbasis = Basis(Tcurr, posvelLMC, potm, potl)
    icbasis = Basis(Tbegin, ic0, potm, potl)
    # Offsets for initial conditions in nonlinearly scaled coordinates - different in each dimension:
    # [0] => positive offset increases orbital phase (i.e orbit will arrive at the target plane earlier);
    # [1] => positive offset decreases orbital energy, making period shorter, i.e. again an earlier arrival;
    # [2] => positive offset decreases angular momentum at fixed energy, with similar consequences;
    # [3-5] - variations in orbital plane and pericentre direction, less important for the overall evolution.
    # These values may need to be adjusted to balance two opposite requirements:
    # on the one hand, too small offsets make it difficult to measure the Jacobian reliably,
    # because the trajectories can only be measured with some noise, which propagates into the Jacobian.
    # On the other hand, too large offsets may lead to very different companion orbits, so that the mapping
    # between initial and final conditions becomes too nonlinear and not well described by a Jacobian.
    offsets = [0.2, 2, 2, 10, 2, 2]
    ndim = 6
    numorbits = 1 + 2*ndim   # base orbit and pairs of companion orbits for each dimension in phase space
    # Note: contrary to the paper, here the IC and FC matrices have been transposed,
    # i.e. each row gives the 6 phase-space coords for one orbit. 
    # The Jacobian is not transposed, i.e. J.dot(ic) = fc  where ic, fc are K column-vectors (shape 6xK),
    # but following numpy conventions, if K=1, these can be 1d vectors of length 6.
    ic = numpy.zeros((numorbits, ndim))
    scaledic = numpy.zeros((numorbits, ndim))
    for w in range(numorbits):
        if w>0:
            scaledic[w, (w-1)//2] = offsets[(w-1)//2] * (1 if w%2 else -1)
        ic[w] = icbasis.fromscaled(scaledic[w])
    #print(ic)

    # simulation end time should be comfortably larger than Tcurr, to leave room for time offsets of orbits
    # and still have them pass through the pericentre within the duration of the simulation
    Tend = Tcurr + 1.0

    # the key part - running the simulation or obtaining the MW and LMC trajectories in some other way;
    # tgrid is an array of length numtimestamps, and rawtrajs has shape (numorbits, numtimestamps, 12),
    # where the first 6 columns are MW pos/vel and the remaining 6 columns are the LMC pos/vel
    tgrid, rawtrajs = getTrajectories(ic, Tbegin, Tend, potm, potl)

    # fit smooth orbits to the raw measured trajectories, construct orbit interpolators,
    # and compute nonlin-scaled final deviations of all orbits from the target point
    smoothtrajs = []
    orbits = numpy.zeros(numorbits, dtype=object)
    scaledfc = numpy.zeros((numorbits, ndim))
    for w in range(numorbits):
        tfitm, tfitl = fitSmoothOrbit(tgrid, rawtrajs[w][:, 0:ndim], rawtrajs[w][:, ndim:2*ndim], potm, potl)
        smoothtrajs.append(numpy.column_stack([tfitm, tfitl]))
        # orbits[w] is a smooth interpolator for the trajectory of the LMC relative to the Milky Way
        orbits[w] = buildOrbit(tgrid, smoothtrajs[w][:, ndim:2*ndim] - smoothtrajs[w][:, 0:ndim])
        scaledfc[w] = fcbasis.toscaled(orbits[w])

    # nonlin-scaled initial offsets from the base orbit
    deltaic = scaledic - scaledic[0]
    # nonlin-scaled final offsets from the base orbit (not from the target!)
    deltafc = scaledfc - scaledfc[0]
    print('#  deviations of all orbits from the target |deviations of|total dev. |offsets from base orbit\n'
          '#  point in the reference hyperplane        |crossing time|from target|initial => final')
    for w in range(numorbits):
        print('[%7.3f, %7.3f, %7.3f, %7.3f, %7.3f], dt=%8.5f  %7.3f   %s' %
              (tuple(scaledfc[w,1:]) + (scaledfc[w,0], sum(scaledfc[w,1:]**2)**0.5,
              ' [BASE ORBIT]' if w==0 else '%5.2f => %5.2f' % 
              (sum(deltaic[w]**2)**0.5, sum(deltafc[w]**2)**0.5))) )

    if numpy.all(numpy.isfinite(scaledfc)):
        print('Method 1: fit the forward jacobian and the current FC, then invert J and predict the next IC')
        # In this method, we determine the (ndim+1) * ndim matrix of free parameters
        # (the final coordinates "xi" of the base orbit and the Jacobian of mapping from initial to final coordinates).
        # Since there are more equations (numorbits * ndim) than unknowns, the linear system is overdetermined
        # and its solution is obtained in the least-square sense.
        # Then we compute the next initial conditions by inverting the Jacobian and multiplying it by -xi.
        matrix = numpy.column_stack([numpy.ones(numorbits), scaledic])  # shape: (numorbits, ndim+1)
        rhs = scaledfc                                                  # shape: (numorbits, ndim)
        # one may impose some additional balancing on the linear system,
        # multiplying each row of the matrix and rhs by its own weighting coefficient
        weights = numpy.ones(numorbits)  # here this option is not used
        sol = numpy.linalg.lstsq(matrix * weights[:,None], rhs * weights[:,None], rcond=None)[0]
        xi  = sol[0]     # row-vector of predicted final coordinates of the base orbit
        J   = sol[1:].T  # Jacobian of transformation from initial to final coordinates
        # predicted next initial coords that should map to zero final coords
        nextscaledic1 = -numpy.linalg.inv(J).dot(xi)
        # convert from scaled to real coords using (approximate!) orbit integration -
        # this becomes less accurate as the time difference (first element of IC vector) increases
        nextrealic1 = icbasis.fromscaled(nextscaledic1)
        # since the solution was approximate, we can compare the actual FC with the one predicted from the Jacobian
        scaledfcpred1 = J.dot(deltaic.T).T + xi
        print('Jacobian and its singular values: \n%s\n%s' % (J, numpy.linalg.svd(J, compute_uv=False)))
        print('Next scaled IC: %s' % nextscaledic1)
        print('Next real IC: %s\n' % numpy.array2string(nextrealic1, separator=',', precision=4))

        print('Method 2: fit the inverse jacobian and the next IC')
        # In this method, we reverse the placement of initial and final coordinates, and the free parameters
        # are the predicted initial conditions of the next orbit and the inverse Jacobian matrix (not used by itself).
        # The advantage is that next IC come directly from the least-square solution without the need to invert
        # the Jacobian, but the downside is that the noisy final coordinates are now part of the design matrix,
        # not the RHS of the least-square fit. In practice, this method gives a somewhat less accurate prediction,
        # but in case that the Jacobian is very ill-determined, the first method may produce a completely unreasonable
        # result, while it is less likely for the second method.
        matrix = numpy.column_stack([numpy.ones(numorbits), scaledfc])  # shape: (numorbits, ndim+1)
        rhs = scaledic                                                  # shape: (numorbits, ndim)
        sol = numpy.linalg.lstsq(matrix, rhs, rcond=None)[0]
        nextscaledic2 = sol[0]
        nextrealic2 = icbasis.fromscaled(nextscaledic2)
        J = numpy.linalg.inv(sol[1:].T)
        scaledfcpred2 = J.dot((scaledic - nextscaledic2).T).T
        print('Jacobian and its singular values: \n%s\n%s' % (J, numpy.linalg.svd(J, compute_uv=False)))
        print('Next scaled IC: %s' % nextscaledic2)
        print('Next real IC: %s\n' % numpy.array2string(nextrealic2, separator=',', precision=4))
    else:  # failed due to trajectories being too far from truth
        nextrealic1, nextrealic2 = numpy.zeros((2, ndim)) * numpy.nan
        scaledfcpred1, scaledfcpred2 = numpy.zeros((2, numorbits, ndim)) * numpy.nan

    # the rest of this routine constructs various diagnostic plots

    # jacobian.pdf
    # first panel shows the Galactocentric distance as a function of time for all orbits;
    # remaining 5 panels show 2d projections of the 6d space of nonlin-scaled final coordinates,
    # with the time deviation on the horizontal axis and each of the 5 remaining coordinates
    # on the vertical axis. The target pos/vel is at origin in all these panels, shown by a black square.
    # The base orbit is shown by a gray circle, and pairs of companion orbits with opposite initial offsets
    # in each dimension - by six different colours, '+' for positive offset and 'x' for negative offset.
    # Ideally, the opposite initial offsets should also stay opposite in the final plane, but this
    # may not happen due to errors in determining the final phase-space point from noisy orbit track,
    # and/or due to nonlinearities in the initial-to-final mapping beyond the first-order Jacobian.
    # To see how well this assumption is satisfied, we plot the "expected" deviation vectors, computed
    # from the fitted Jacobian and therefore symmetric w.r.t. the base orbit, with open circles (method 1).
    # Likewise, expected deviations from method 2 are plotted by open squares, and the base orbit may also
    # be displaced from its expectation. If both squares and circles align well with actual points, this
    # indicates that the jacobian and the prediction for the next IC are likely reliable; otherwise it may
    # help to choose which of the two methods is more likely to give sensible prediction for the next IC.
    colors = ['b', 'g', 'r', '#a0a000', 'm', '#00a0a0']
    axes = plt.subplots(2, 3, figsize=(15,10))[1].reshape(-1)
    axes[0].set_xlabel('time')
    axes[0].set_ylabel('distance')
    # determine the range of actual deviations to set the axes limits,
    # including the target (origin), but ignoring expected deviations, which might be wildly off..
    mindev, maxdev = numpy.nanpercentile(numpy.vstack([numpy.zeros(ndim), scaledfc]), [0,100], axis=0)
    middev = (maxdev+mindev)/2
    drange = (maxdev-mindev)/2 * 1.2
    for a in range(1,ndim):
        axes[a].plot(0, 0, 's', c='w', mew=3, mec='k')
        axes[a].set_xlabel('delta t')
        axes[a].set_ylabel('delta fc[%i]' % a)
        axes[a].set_xlim(middev[0]-drange[0], middev[0]+drange[0])
        axes[a].set_ylim(middev[a]-drange[a], middev[a]+drange[a])
        axes[a].plot([0.0], [1.0], 'o', ms=30, mew=0, color=colors[a], alpha=0.5, transform=axes[a].transAxes)
    for w in range(numorbits):
        if w==0:
            col = 'gray'
            marker='o'
        else:
            deltafc[w-1] = scaledfc[w] - scaledfc[0]
            col = colors[(w-1)//2]
            marker = '+' if w%2 else 'x'
        axes[0].plot(tgrid, numpy.sum((smoothtrajs[w][:, ndim:ndim+3] - smoothtrajs[w][:, 0:3])**2, axis=1)**0.5,
            color=col, dashes=[1000,1] if w%2 else [3,2], lw=0.5)
        for a in range(1,ndim):
            axes[a].plot(scaledfc[w,0], scaledfc[w,a], marker=marker, color=col, mec=col, mew=2)
            axes[a].plot(scaledfcpred1[w,0], scaledfcpred1[w,a], marker='o', color='none', mec=col, alpha=0.3)
            axes[a].plot(scaledfcpred2[w,0], scaledfcpred2[w,a], marker='s', color='none', mec=col, alpha=0.3)
    #plt.savefig('jacobian.pdf')

    # orbitfit.pdf
    # Gauge the accuracy and possible biases of smooth orbit approximations.
    # Each column shows the deviations between raw (measured) and smooth (fitted) orbits:
    # positions in the first row, velocities in the second row.
    # If there are noticeable coherent residual deviations, this suggests that the grid for B-spline fits
    # in fitSmoothOrbit() should be denser.
    # The remaining rows show the accelerations (second derivatives of the trajectories),
    # total acc in the third row, and residual (fitted) acceleration in the fourth row.
    # If the latter one is too wiggly, this suggests that the B-spline grid is too dense and prone to overfitting.
    axes = plt.subplots(4, numorbits, figsize=(30,10))[1]
    for w in range(numorbits):
        checkSmoothOrbit(axes[:,w], tgrid, rawtrajs[w][:, 0:ndim], rawtrajs[w][:, ndim:2*ndim],
                         smoothtrajs[w][:, 0:ndim], smoothtrajs[w][:, ndim:2*ndim], potm, potl)
    for r in range(4):
        axes[r,0].get_shared_y_axes().join(*tuple(axes[r,1:]))
        axes[r,0].legend(loc='upper left', ncol=2, fontsize=8)
        for c in range(1, len(ic)):
            axes[r,c].set_ylabel('')
            axes[r,c].set_yticklabels([])
    plt.tight_layout()
    #plt.savefig('orbitfit.pdf')

    # return predicted next initial conditions from both methods, and let the user choose
    return nextrealic1, nextrealic2

### Trajectory computation

In [ ]:
def fakeOrbit(ic, Tic, Tbegin, Tend, potm, potl, Tstep = 1./128, adddrag=True):
    '''
    Function for producing somewhat realistic (though not very accurate) fake orbits of both galaxies,
    using the approximate equations of motion of two extended bodies plus dynamical friction for the LMC.
    Arguments:
        ic:     array of length 6 - the initial position and velocity of the LMC in the Galactocentric frame;
        Tic:    timestamp associated with these IC;
        Tbegin: earliest timestamp on the returned orbit;
        Tend:   latest timestamp on the returned orbit,
                Tic should be somewhere between the two, but not necessarily at one of the endpoints;
        potm:   Milky Way potential;
        potl:   LMC potential.
    Return:
        three arrays: timestamps (length M), trajectory of the Milky Way (shape Mx6) and LMC (same).
    '''
      # output timestep - somewhat arbitrary, may be adjusted at will
    tgrid = numpy.linspace(Tbegin, Tend, round((Tend-Tbegin) / Tstep) + 1)
    indic = numpy.searchsorted(tgrid, Tic)
    def difeq(vars, t):
        x0    = vars[0:3]            # MW position
        v0    = vars[3:6]            # MW velocity
        x1    = vars[6:9]            # LMC position
        v1    = vars[9:12]           # LMC velocity
        dx    = x1-x0                # relative offset
        dv    = v1-v0                # relative velocity
        dist  = sum(dx**2)**0.5      # distance between the galaxies
        vmag  = sum(dv**2)**0.5      # magnitude of relative velocity
        f0    = potl.force(-dx, t=t) # force from LMC acting on the MW center
        f1    = potm.force( dx, t=t) # force from MW acting on the LMC
        rho   = potm.density(dx,t=t) # actual MW density at this point
        sigma = sigmafnc(dist)       # approximate MW velocity dispersion at this point
        mass  = massfnc(t)           # time-dependent LMC mass
        # distance-dependent Coulomb logarithm (Hashimoto+ 2003)
        couLog= max(0, numpy.log(dist / bminCouLog))
        X     = vmag / (sigma * 2**.5)
        if adddrag:
            drag  = -(4*numpy.pi * rho * dv / vmag *
                (scipy.special.erf(X) - 2/numpy.pi**.5 * X * numpy.exp(-X*X)) *
                mass * agama.G**2 / vmag**2 * couLog)   # dynamical friction force
        else:
            drag = 0
        return numpy.hstack((v0, f0, v1, f1 + drag))

    # LMC mass for dynamical friction may vary with time, here assumed const
    massfnc  = lambda t: potl_params['mass']/2
    bminCouLog = potl_params['scaleRadius'] * 0.8 # lower cutoff for the Coulomb logarithm
    ic   = numpy.hstack((numpy.zeros(6), ic))
    if indic > 0:
        sol1 = scipy.integrate.odeint(difeq, ic, tgrid[:indic+1][::-1])[::-1]
    else:
        sol1 = ic.copy()
    if indic < len(tgrid)-1:
        sol2 = scipy.integrate.odeint(difeq, ic, tgrid[indic:])[1:]
    else:
        sol2 = numpy.zeros((0,12))
    sol  = numpy.vstack([sol1, sol2])
    return tgrid, sol[:,0:6], sol[:,6:12]

In [ ]:
def getTrajectories(ic, Tbegin, Tend, potm, potl, noise=1.0, plot=True):
    '''
    A function that should run a bunch of N-body simulations with the given initial conditions for the LMC
    and produce an array of "raw" (noisy) trajectories of both galaxies.
    For demonstration purposes, we replace the actual N-body simulation with fake orbit computation,
    and add a somewhat realistic amount of noise to measured position and velocity of centres of both galaxies.
    Arguments:
        ic:     array of Kx6 initial conditions for orbits;
        Tbegin: starting time for the simulation;
        Tend:   end time for the simulation (should be later than the present time);
        potm:   potential of the Milky Way;
        potl:   potential of the LMC (both needed to fake the orbits; in reality the N-body simulation
                would need just the initial snapshots of both galaxies);
        noise:  add a given amount of noise to computed orbits, to mimic the noise in N-body sims;
        plot:   whether to show a diagnostic plot for fitting smoothed orbits.
    Return:
        two arrays - timestamps (length M) and orbits (K arrays of shape Mx12 representing
        trajectories of the Milky Way (first 6 columns) and the LMC (remaining 6 columns)
        in the inertial reference frame (e.g. centre of mass frame).
    '''
    trajs = []
    for w in range(len(ic)):
        tgrid, trajm, trajl = fakeOrbit(ic[w], Tbegin, Tbegin, Tend, potm, potl)
        # add some noise with a deterministic seed
        numpy.random.seed(round(abs(trajl[-1,0])*100000))
        trajm += numpy.random.normal(size=trajm.shape) * numpy.array([0.2,0.2,0.2,1.,1.,1.]) * noise
        trajl += numpy.random.normal(size=trajl.shape) * numpy.array([0.2,0.2,0.2,1.,1.,1.]) * noise
        trajs.append(numpy.column_stack([trajm, trajl]))
    return tgrid, trajs

### Main program

In [ ]:
agama.getUnits()

In [ ]:
# Initial setup:

# present-day position and velocity of the LMC in Galactocentric coordinates
posvelLMC = numpy.array([  -0.61,  -41.02,  -26.83,  -69.84,  -221.66,  214.12 ])
# MW potential
potm_params = [  # bulge, disk, halo
    dict(type='Spheroid', mass=1.2e10, scaleRadius=0.2, outerCutoffRadius=1.8, gamma=0, beta=1.8),
    dict(type='Disk', surfaceDensity=0.088431375e10, scaleRadius=3.0, scaleHeight=-0.25),
    dict(type='Spheroid', densityNorm=0.97e7, scaleRadius=16.5, outerCutoffRadius=500, cutoffStrength=4, beta=3, gamma=1)
]
potm = agama.Potential(*potm_params)

# MW halo velocity dispersion profile
gridr = numpy.logspace(1, 3, 16)
potm_halo = agama.Potential(potm_params[2])
sigmas = agama.GalaxyModel(potm_halo, agama.DistributionFunction(type='quasispherical', potential=potm_halo)).moments(numpy.column_stack((gridr, gridr*0, gridr*0)), dens=False, vel=False)[:,0]**0.5

sigmafnc = agama.Spline(gridr, sigmas)


# LMC potential
potl_params = dict(type='spheroid', mass=2e11, scaleRadius=9.0, outerCutoffRadius=90, cutoffStrength=4, beta=3)
potl = agama.Potential(potl_params)
# initial time for the simulation (in time units of kpc/(km/s), which is approximately 1 Gyr)

Tbegin = -10.0

# "true" initial conditions obtained by using our fakeOrbit() routine
tt, trajmtrue, trajltrue = fakeOrbit(posvelLMC, 0.0, Tbegin, 0.0, potm, potl, adddrag=True, Tstep=1e-4)
ictrue = trajltrue[0] - trajmtrue[0]
print('True IC: %s' % ictrue)

In [ ]:
traj = trajltrue - trajmtrue

In [ ]:
x, y, z = traj[:, 0], traj[:, 1], traj[:, 2]

In [ ]:
plt.plot(y, z)
plt.gca().set_aspect(1)

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(
dict(
    t = tt,
    x = x, 
    y = y,
    z = z,
    vx = traj[:, 3],
    vy = traj[:, 4],
    vz = traj[:, 5],
))

In [ ]:
df.to_csv("lmc_orbit.csv")

In [ ]:
pwd

In [ ]:
r = numpy.sqrt(x**2 + y**2 + z**2)
numpy.min(r), numpy.max(r)

In [ ]:
plt.plot(tt, r)

## L3

In [ ]:
# Initial setup:

# present-day position and velocity of the LMC in Galactocentric coordinates
posvelLMC = numpy.array([  -0.61,  -41.02,  -26.83,  -69.84,  -221.66,  214.12 ])
# MW potential
potm_params = [  # bulge, disk, halo
    dict(type='Spheroid', mass=1.2e10, scaleRadius=0.2, outerCutoffRadius=1.8, gamma=0, beta=1.8),
    dict(type='Disk', surfaceDensity=0.088431375e10, scaleRadius=3.0, scaleHeight=-0.25),
    dict(type='Spheroid', densityNorm=0.97e7, scaleRadius=16.5, outerCutoffRadius=500, cutoffStrength=4, beta=3, gamma=1)
]
potm = agama.Potential(*potm_params)

# MW halo velocity dispersion profile
gridr = numpy.logspace(1, 3, 16)
potm_halo = agama.Potential(potm_params[2])
sigmas = agama.GalaxyModel(potm_halo, agama.DistributionFunction(type='quasispherical', potential=potm_halo)).moments(numpy.column_stack((gridr, gridr*0, gridr*0)), dens=False, vel=False)[:,0]**0.5

sigmafnc = agama.Spline(gridr, sigmas)


# LMC potential
potl_params = dict(type='spheroid', mass=3e11, scaleRadius=11.7, outerCutoffRadius=117, cutoffStrength=4, beta=3)
potl = agama.Potential(potl_params)
# initial time for the simulation (in time units of kpc/(km/s), which is approximately 1 Gyr)

Tbegin = -10.0

# "true" initial conditions obtained by using our fakeOrbit() routine
tt, trajmtrue, trajltrue = fakeOrbit(posvelLMC, 0.0, Tbegin, 0.0, potm, potl, adddrag=True, Tstep=1e-4)
ictrue = trajltrue[0] - trajmtrue[0]
print('True IC: %s' % ictrue)

In [ ]:
traj = trajltrue - trajmtrue

In [ ]:
x, y, z = traj[:, 0], traj[:, 1], traj[:, 2]

In [ ]:
potm_halo.totalMass() / 1e10

In [ ]:
sigmas

In [ ]:
plt.plot(y, z)
plt.gca().set_aspect(1)

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(
dict(
    t = tt,
    x = x, 
    y = y,
    z = z,
    vx = traj[:, 3],
    vy = traj[:, 4],
    vz = traj[:, 5],
))

In [ ]:
df.to_csv("L3M11_orbit.csv")

In [ ]:
pwd

In [ ]:
r = numpy.sqrt(x**2 + y**2 + z**2)
numpy.min(r), numpy.max(r)

In [ ]:
plt.plot(tt, r)

## L3M10

In [ ]:
# Initial setup:

# present-day position and velocity of the LMC in Galactocentric coordinates
posvelLMC = numpy.array([  -0.61,  -41.02,  -26.83,  -69.84,  -221.66,  214.12 ])
# MW potential
potm_params = [  # bulge, disk, halo
    dict(type='Spheroid', mass=1.2e10, scaleRadius=0.2, outerCutoffRadius=1.8, gamma=0, beta=1.8),
    dict(type='Disk', surfaceDensity=0.088431375e10, scaleRadius=3.0, scaleHeight=-0.25),
    dict(type='Spheroid', densityNorm=0.97e7, scaleRadius=16.5, outerCutoffRadius=500, cutoffStrength=4, beta=3, gamma=1)
]
potm = agama.Potential(*potm_params)

# MW halo velocity dispersion profile
gridr = numpy.logspace(1, 3, 16)
potm_halo = agama.Potential(potm_params[2])
sigmas = agama.GalaxyModel(potm_halo, agama.DistributionFunction(type='quasispherical', potential=potm_halo)).moments(numpy.column_stack((gridr, gridr*0, gridr*0)), dens=False, vel=False)[:,0]**0.5

sigmafnc = agama.Spline(gridr, sigmas)


# LMC potential
potl_params = dict(type='spheroid', mass=3e11, scaleRadius=11.7, outerCutoffRadius=117, cutoffStrength=4, beta=3)
potl = agama.Potential(potl_params)
# initial time for the simulation (in time units of kpc/(km/s), which is approximately 1 Gyr)

Tbegin = -10.0

# "true" initial conditions obtained by using our fakeOrbit() routine
tt, trajmtrue, trajltrue = fakeOrbit(posvelLMC, 0.0, Tbegin, 0.0, potm, potl, adddrag=True, Tstep=1e-4)
ictrue = trajltrue[0] - trajmtrue[0]
print('True IC: %s' % ictrue)

In [ ]:
traj = trajltrue - trajmtrue

In [ ]:
x, y, z = traj[:, 0], traj[:, 1], traj[:, 2]

In [ ]:
potm_halo.totalMass() / 1e10

In [ ]:
plt.plot(y, z)
plt.gca().set_aspect(1)

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(
dict(
    t = tt,
    x = x, 
    y = y,
    z = z,
    vx = traj[:, 3],
    vy = traj[:, 4],
    vz = traj[:, 5],
))

In [ ]:
df.to_csv("L3M11_orbit.csv")

In [ ]:
pwd

In [ ]:
r = numpy.sqrt(x**2 + y**2 + z**2)
numpy.min(r), numpy.max(r)

In [ ]:
plt.plot(tt, r)